### Please provide your own S3 bucket below. The name for your bucket must contain the prefix ‘deeplens’. In this example, the bucket is ‘deeplens-imageclassification’. Make Sure S3 bucket name is unique, e.g. deeplens-imageclassfication-name-date

In [51]:
bucket='deeplens-imageclassification-matthew'

### 获取笔记本实例的执行角色。这是您在创建笔记本实例时创建的 IAM 角色。将角色传递给调优作业。

In [52]:
from sagemaker import get_execution_role
role = get_execution_role()

In [53]:
#Using predefine Resnet model
training_image = '811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:latest'
# The algorithm supports multiple network depth (number of layers). They are 18, 34, 50, 101, 152 and 200
# For this training, we will use 18 layers
num_layers = "50" 


In [54]:
# we need to specify the input image shape for the training data
image_shape = "3,224,224"
# we also need to specify the number of training samples in the training set
# for caltech it is 15420
num_training_samples = "22504"
# specify the number of output classes
num_classes = "3"

### Please make sure number set for epochs is same as checkpoint_frequency

In [55]:
# number of epochs
epochs = "60"

In [56]:
# batch size for training
mini_batch_size =  "50"
# learning rate
learning_rate = "0.1"
#optimizer
#optimizer ='Adam'
#checkpoint_frequency
checkpoint_frequency = "3"
#scheduler_step
lr_scheduler_step="30,90,180"
#scheduler_factor
lr_scheduler_factor="0.1"

In [57]:
InstanceType="ml.p3.16xlarge"

In [58]:
#augmentation_type
augmentation_type="crop_color_transform"

In [59]:
# create unique job name 
import time
from time import gmtime, strftime
job_name_prefix = 'DEMO-Matthew-kaggle-dog-cat-classification'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

### Please make job_name_prefix below unique so that its easy to remember in Deeplens projects.
### InstanceType below is using ml.p3.2xlarge but if you dont have these instances in your account then you can use non GPU instaces such as ml.c5.xlarge. please check https://aws.amazon.com/sagemaker/pricing/instance-types for more different instance types available for Sagemaker

In [60]:
%%time
import pickle

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

training_params = \
{
    # specify the training docker image
    "AlgorithmSpecification": {
        "TrainingImage": training_image,
        "TrainingInputMode": "File"
    },
    "RoleArn": role,
    "OutputDataConfig": {
        "S3OutputPath": 's3://{}/{}/output'.format(bucket, job_name_prefix)
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": InstanceType,
        "VolumeSizeInGB": 50
    },
    "TrainingJobName": job_name,
    "HyperParameters": {
        "image_shape": image_shape,
        "num_layers": str(num_layers),
        "num_training_samples": str(num_training_samples),
        "num_classes": str(num_classes),
        "mini_batch_size": str(mini_batch_size),
        "epochs": str(epochs),
        "learning_rate": str(learning_rate),
        "lr_scheduler_step": str(lr_scheduler_step),
        "lr_scheduler_factor": str(lr_scheduler_factor),
        "augmentation_type": str(augmentation_type),
        "checkpoint_frequency": str(checkpoint_frequency),
        "augmentation_type" : str(augmentation_type)
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 360000
    },
#Training data should be inside a subdirectory called "train"
#Validation data should be inside a subdirectory called "validation"
#The algorithm currently only supports fullyreplicated model (where data is copied onto each machine)
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": 's3://{}/kaggle/train/'.format(bucket),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "application/x-recordio",
            "CompressionType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": 's3://{}/kaggle/validation/'.format(bucket),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "application/x-recordio",
            "CompressionType": "None"
        }
    ]
}

save_obj(training_params, "training_params_kaggle" )

print('Training job name: {}'.format(job_name))
print('\nInput Data Location: {}'.format(training_params['InputDataConfig'][0]['DataSource']['S3DataSource']))

Training job name: DEMO-Matthew-kaggle-dog-cat-classification-2020-05-14-14-47-48

Input Data Location: {'S3DataType': 'S3Prefix', 'S3Uri': 's3://deeplens-imageclassification-matthew/kaggle/train/', 'S3DataDistributionType': 'FullyReplicated'}
CPU times: user 500 µs, sys: 64 µs, total: 564 µs
Wall time: 460 µs
